In [1]:
import timeit
import os
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture as GMM
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.random_projection import SparseRandomProjection
from sklearn.ensemble import RandomForestClassifier



from helpers import get_abspath
import warnings
warnings.filterwarnings('ignore')
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])


In [2]:
train_df = pd.read_csv('../data/optdigits_train.csv', header=None)
digits_y = train_df.iloc[:, -1:].as_matrix().flatten()
digits_X = train_df.iloc[:, :-1].as_matrix()

train_df = pd.read_csv('../data/abalone_train.csv', header=None)
abalone_y = train_df.iloc[:, -1:].as_matrix().flatten()
abalone_X = train_df.iloc[:, :-1].as_matrix()

In [3]:
km = KMeans(random_state=0)  # K-Means
gmm = GMM(random_state=0)  # Gaussian Mixture Model (EM)
km.set_params(n_clusters=3)
gmm.set_params(n_components=11)
km.fit(abalone_X)
gmm.fit(abalone_X)
cols = ["Length", "Diameter", "Height", "Whole weight", "Shucked weight", "Viscera weight", "Shell weight", "Male", "Female", "Infant"]

df_gmm = pd.DataFrame(columns=cols, data=abalone_X)
df_km = pd.DataFrame(columns=cols, data=abalone_X)
df_gmm['cluster'] = gmm.predict(abalone_X)
df_km['cluster'] = km.predict(abalone_X)


In [4]:
df = df_gmm.groupby('cluster')['Male','Female','Infant'].agg(['sum'])
df.columns= ["Male", "Female", "Infant"]
df.T

cluster,0,1,2,3,4,5,6,7,8,9,10
Male,259.0,0.0,0.0,631.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0
Female,0.0,279.0,0.0,0.0,193.0,0.0,0.0,0.0,0.0,382.0,74.0
Infant,0.0,0.0,441.0,0.0,0.0,109.0,126.0,0.0,389.0,0.0,0.0


In [5]:
df = df_km.groupby('cluster')['Male','Female','Infant'].agg(['sum'])
df.columns= ["Male", "Female", "Infant"]
df.T

cluster,0,1,2
Male,930.0,0.0,0.0
Female,1.0,927.0,0.0
Infant,0.0,0.0,1065.0


In [6]:
test_df = pd.read_csv('../data/optdigits_test.csv', header=None)
digits_y_test = test_df.iloc[:, -1:].as_matrix().flatten()
digits_X_test = test_df.iloc[:, :-1].as_matrix()

test_df = pd.read_csv('../data/abalone_test.csv', header=None)
abalone_y_test = test_df.iloc[:, -1:].as_matrix().flatten()
abalone_X_test = test_df.iloc[:, :-1].as_matrix()

In [7]:
digits_scalar = StandardScaler()

digits_scalar.fit(digits_X)
digits_X = digits_scalar.transform(digits_X)
digits_X_test = digits_scalar.transform(digits_X_test)

In [8]:
abalone_scalar = StandardScaler()

abalone_scalar.fit(abalone_X)
abalone_X = abalone_scalar.transform(abalone_X)
abalone_X_test = abalone_scalar.transform(abalone_X_test)

In [9]:
## BASELINE
ann = MLPClassifier(
    activation='relu', max_iter=5000,
    solver='adam', learning_rate='adaptive', 
    hidden_layer_sizes=(500, 500), alpha= 0.01
)

start_time = timeit.default_timer()
ann.fit(digits_X,digits_y)
predict = ann.predict(digits_X_test)
print(classification_report(predict, digits_y_test))
end_time = timeit.default_timer()
elapsed = end_time - start_time
print("Done: {} seconds".format(elapsed))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       178
          1       0.99      0.95      0.97       191
          2       0.98      0.98      0.98       176
          3       0.95      0.99      0.97       175
          4       0.98      0.98      0.98       182
          5       0.99      0.96      0.97       188
          6       0.99      0.99      0.99       180
          7       0.93      0.98      0.96       170
          8       0.92      0.96      0.94       166
          9       0.97      0.92      0.94       191

avg / total       0.97      0.97      0.97      1797

Done: 27.17330858600326 seconds


In [10]:
km = KMeans(random_state=0)  # K-Means
gmm = GMM(random_state=0)  # Gaussian Mixture Model (EM)
km.set_params(n_clusters=10)
gmm.set_params(n_components=8)
km.fit(digits_X)
gmm.fit(digits_X)
digits_X_km = np.concatenate(
    (digits_X, get_one_hot(km.predict(digits_X), 10)),
    axis=1
)
digits_X_gmm = np.concatenate(
    (digits_X, get_one_hot(gmm.predict(digits_X), 8)),
    axis=1
)
digits_X_test_km = np.concatenate(
    (digits_X_test, get_one_hot(km.predict(digits_X_test), 10)),
    axis=1
)
digits_X_test_gmm = np.concatenate(
    (digits_X_test, get_one_hot(gmm.predict(digits_X_test), 8)),
    axis=1
)

In [11]:
##KMEANS
ann = MLPClassifier(
    activation='relu', max_iter=5000,
    solver='adam', learning_rate='adaptive', 
    hidden_layer_sizes=(500, 500), alpha= 0.01
)
start_time = timeit.default_timer()
ann.fit(digits_X_gmm,digits_y)
predict = ann.predict(digits_X_test_gmm)
print(classification_report(predict, digits_y_test))
end_time = timeit.default_timer()
elapsed = end_time - start_time
print("Done: {} seconds".format(elapsed))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99       176
          1       0.97      0.97      0.97       182
          2       0.98      0.97      0.98       179
          3       0.96      0.99      0.98       178
          4       0.99      0.96      0.98       186
          5       0.99      0.94      0.97       193
          6       0.98      0.99      0.99       180
          7       0.93      0.99      0.96       168
          8       0.94      0.95      0.94       172
          9       0.95      0.93      0.94       183

avg / total       0.97      0.97      0.97      1797

Done: 22.981471509905532 seconds


In [12]:
##GMM
ann = MLPClassifier(
    activation='relu', max_iter=5000,
    solver='adam', learning_rate='adaptive', 
    hidden_layer_sizes=(500, 500), alpha= 0.01
)
start_time = timeit.default_timer()
ann.fit(digits_X_km,digits_y)
predict = ann.predict(digits_X_test_km)
print(classification_report(predict, digits_y_test))
end_time = timeit.default_timer()
elapsed = end_time - start_time
print("Done: {} seconds".format(elapsed))

             precision    recall  f1-score   support

          0       1.00      0.99      1.00       179
          1       0.99      0.96      0.98       187
          2       0.99      0.98      0.99       178
          3       0.97      0.99      0.98       178
          4       0.97      0.99      0.98       178
          5       0.99      0.95      0.97       191
          6       0.99      1.00      0.99       179
          7       0.92      1.00      0.96       165
          8       0.94      0.94      0.94       174
          9       0.97      0.93      0.95       188

avg / total       0.97      0.97      0.97      1797

Done: 23.504933387041092 seconds


# PCA 

In [13]:
pca = PCA(random_state=0, svd_solver='full', n_components=32)
pca.fit(digits_X)
digits_X_PCA = pca.transform(digits_X)
digits_X_test_PCA = pca.transform(digits_X_test)

In [14]:
ann = MLPClassifier(
    activation='relu', max_iter=5000,
    solver='adam', learning_rate='adaptive', 
    hidden_layer_sizes=(500, 500), alpha= 0.01
)
start_time = timeit.default_timer()
ann.fit(digits_X_PCA,digits_y)
predict = ann.predict(digits_X_test_PCA)
print(classification_report(predict, digits_y_test))
end_time = timeit.default_timer()
elapsed = end_time - start_time
print("Done: {} seconds".format(elapsed))

             precision    recall  f1-score   support

          0       1.00      0.99      1.00       179
          1       0.99      0.92      0.95       196
          2       0.97      0.98      0.97       175
          3       0.95      0.99      0.97       175
          4       0.97      0.94      0.96       187
          5       0.98      0.95      0.96       187
          6       0.98      0.98      0.98       181
          7       0.91      0.99      0.95       164
          8       0.91      0.96      0.94       166
          9       0.93      0.90      0.92       187

avg / total       0.96      0.96      0.96      1797

Done: 26.210280172992498 seconds


In [15]:
km = KMeans(random_state=0)  # K-Means
gmm = GMM(random_state=0)  # Gaussian Mixture Model (EM)
km.set_params(n_clusters=20)
gmm.set_params(n_components=12)
km.fit(digits_X_PCA)
gmm.fit(digits_X_PCA)
digits_X_PCA_km = np.concatenate(
    (digits_X_PCA, get_one_hot(km.predict(digits_X_PCA), 20)),
    axis=1
)
digits_X_PCA_gmm = np.concatenate(
    (digits_X_PCA, get_one_hot(gmm.predict(digits_X_PCA), 12)),
    axis=1
)
digits_X_test_PCA_km = np.concatenate(
    (digits_X_test_PCA, get_one_hot(km.predict(digits_X_test_PCA), 20)),
    axis=1
)
digits_X_test_PCA_gmm = np.concatenate(
    (digits_X_test_PCA, get_one_hot(gmm.predict(digits_X_test_PCA), 12)),
    axis=1
)

In [16]:
## PCA + KMEANS
ann = MLPClassifier(
    activation='relu', max_iter=5000,
    solver='adam', learning_rate='adaptive', 
    hidden_layer_sizes=(500, 500), alpha= 0.01
)
start_time = timeit.default_timer()
ann.fit(digits_X_PCA_km,digits_y)
predict = ann.predict(digits_X_test_PCA_km)
print(classification_report(predict, digits_y_test))
end_time = timeit.default_timer()
elapsed = end_time - start_time
print("Done: {} seconds".format(elapsed))

             precision    recall  f1-score   support

          0       1.00      0.99      1.00       179
          1       0.98      0.94      0.96       191
          2       0.96      0.98      0.97       173
          3       0.96      0.98      0.97       178
          4       0.97      0.97      0.97       181
          5       0.97      0.96      0.97       184
          6       0.99      0.97      0.98       184
          7       0.92      0.99      0.96       166
          8       0.93      0.96      0.94       168
          9       0.96      0.90      0.93       193

avg / total       0.96      0.96      0.96      1797

Done: 23.11047149822116 seconds


In [17]:
## PCA + GMM
ann = MLPClassifier(
    activation='relu', max_iter=5000,
    solver='adam', learning_rate='adaptive', 
    hidden_layer_sizes=(500, 500), alpha= 0.01
)
start_time = timeit.default_timer()

ann.fit(digits_X_PCA_gmm,digits_y)

predict = ann.predict(digits_X_test_PCA_gmm)

print(classification_report(predict, digits_y_test))
end_time = timeit.default_timer()
elapsed = end_time - start_time
print("Done: {} seconds".format(elapsed))

             precision    recall  f1-score   support

          0       0.99      1.00      1.00       177
          1       0.99      0.94      0.97       191
          2       0.96      0.98      0.97       173
          3       0.95      0.98      0.96       178
          4       0.98      0.96      0.97       185
          5       0.98      0.96      0.97       186
          6       0.99      0.98      0.99       182
          7       0.93      0.99      0.96       168
          8       0.95      0.97      0.96       170
          9       0.96      0.92      0.94       187

avg / total       0.97      0.97      0.97      1797

Done: 30.819543289020658 seconds


## ICA

In [18]:
ica = FastICA(random_state=0, max_iter=5000, tol=1e-04, n_components=22)
ica.fit(digits_X)
digits_X_ICA = ica.transform(digits_X)
digits_X_test_ICA = ica.transform(digits_X_test)

In [19]:
ann = MLPClassifier(
    activation='relu', max_iter=5000,
    solver='adam', learning_rate='adaptive', 
    hidden_layer_sizes=(500, 500), alpha= 0.01
)
start_time = timeit.default_timer()
ann.fit(digits_X_ICA,digits_y)
predict = ann.predict(digits_X_test_ICA)
print(classification_report(predict, digits_y_test))
end_time = timeit.default_timer()
elapsed = end_time - start_time
print("Done: {} seconds".format(elapsed))

             precision    recall  f1-score   support

          0       0.97      1.00      0.99       173
          1       0.94      0.86      0.90       199
          2       0.94      0.95      0.95       174
          3       0.89      0.99      0.94       163
          4       0.96      0.93      0.94       188
          5       0.98      0.87      0.92       206
          6       0.98      0.96      0.97       185
          7       0.93      0.97      0.95       172
          8       0.87      0.88      0.88       171
          9       0.84      0.92      0.88       166

avg / total       0.93      0.93      0.93      1797

Done: 8.090884200995788 seconds


In [20]:
km = KMeans(random_state=0)  # K-Means
gmm = GMM(random_state=0)  # Gaussian Mixture Model (EM)
km.set_params(n_clusters=20)
gmm.set_params(n_components=12)
km.fit(digits_X_ICA)
gmm.fit(digits_X_ICA)

digits_X_ICA_km = np.concatenate(
    (digits_X_ICA, get_one_hot(km.predict(digits_X_ICA), 20)),
    axis=1
)
digits_X_ICA_gmm = np.concatenate(
    (digits_X_ICA, get_one_hot(gmm.predict(digits_X_ICA), 12)),
    axis=1
)

digits_X_test_ICA_km = np.concatenate(
    (digits_X_test_ICA, get_one_hot(km.predict(digits_X_test_ICA), 20)),
    axis=1
)
digits_X_test_ICA_gmm = np.concatenate(
    (digits_X_test_ICA, get_one_hot(gmm.predict(digits_X_test_ICA), 12)),
    axis=1
)

In [21]:
## ICA + KMEANS
ann = MLPClassifier(
    activation='relu', max_iter=5000,
    solver='adam', learning_rate='adaptive', 
    hidden_layer_sizes=(500, 500), alpha= 0.01
)
start_time = timeit.default_timer()
ann.fit(digits_X_ICA_km, digits_y)
predict = ann.predict(digits_X_test_ICA_km)
print(classification_report(predict, digits_y_test))
end_time = timeit.default_timer()
elapsed = end_time - start_time
print("Done: {} seconds".format(elapsed))

             precision    recall  f1-score   support

          0       0.98      0.99      0.99       175
          1       0.97      0.83      0.90       212
          2       0.89      0.96      0.92       165
          3       0.87      0.96      0.91       167
          4       0.94      0.93      0.94       183
          5       0.98      0.87      0.92       206
          6       0.96      0.96      0.96       180
          7       0.92      0.98      0.95       167
          8       0.87      0.86      0.87       176
          9       0.84      0.92      0.88       166

avg / total       0.93      0.92      0.92      1797

Done: 16.59419621201232 seconds


In [22]:
## ICA + GMM
ann = MLPClassifier(
    activation='relu', max_iter=5000,
    solver='adam', learning_rate='adaptive', 
    hidden_layer_sizes=(500, 500), alpha= 0.01
)
start_time = timeit.default_timer()
ann.fit(digits_X_ICA_gmm, digits_y)
predict = ann.predict(digits_X_test_ICA_gmm)
print(classification_report(predict, digits_y_test))
end_time = timeit.default_timer()
elapsed = end_time - start_time
print("Done: {} seconds".format(elapsed))

             precision    recall  f1-score   support

          0       0.99      0.99      0.99       178
          1       0.96      0.88      0.92       198
          2       0.97      0.94      0.95       182
          3       0.88      0.98      0.93       165
          4       0.98      0.95      0.97       187
          5       0.98      0.91      0.94       196
          6       0.97      0.95      0.96       185
          7       0.93      0.97      0.95       171
          8       0.87      0.89      0.88       171
          9       0.86      0.95      0.90       164

avg / total       0.94      0.94      0.94      1797

Done: 10.956296659074724 seconds


## RP

In [23]:
rp = SparseRandomProjection(random_state=0, n_components=29)

rp.fit(digits_X)

digits_X_RP = rp.transform(digits_X)
digits_X_test_RP = rp.transform(digits_X_test)

In [24]:
ann = MLPClassifier(
    activation='relu', max_iter=5000,
    solver='adam', learning_rate='adaptive', 
    hidden_layer_sizes=(500, 500), alpha= 0.01
)
start_time = timeit.default_timer()
ann.fit(digits_X_RP,digits_y)
predict = ann.predict(digits_X_test_RP)
print(classification_report(predict, digits_y_test))
end_time = timeit.default_timer()
elapsed = end_time - start_time
print("Done: {} seconds".format(elapsed))

             precision    recall  f1-score   support

          0       0.99      0.99      0.99       178
          1       0.98      0.92      0.95       195
          2       0.97      0.98      0.97       175
          3       0.92      0.96      0.94       175
          4       0.94      0.95      0.94       179
          5       0.97      0.94      0.95       187
          6       0.98      0.96      0.97       186
          7       0.94      0.99      0.97       169
          8       0.90      0.90      0.90       173
          9       0.94      0.94      0.94       180

avg / total       0.95      0.95      0.95      1797

Done: 23.506456583039835 seconds


In [25]:
km = KMeans(random_state=0)  # K-Means
gmm = GMM(random_state=0)  # Gaussian Mixture Model (EM)
km.set_params(n_clusters=10)
gmm.set_params(n_components=9)
km.fit(digits_X_RP)
gmm.fit(digits_X_RP)

digits_X_RP_km = np.concatenate(
    (digits_X_RP, get_one_hot(km.predict(digits_X_RP), 10)),
    axis=1
)
digits_X_RP_gmm = np.concatenate(
    (digits_X_RP, get_one_hot(gmm.predict(digits_X_RP), 9)),
    axis=1
)

digits_X_test_RP_km = np.concatenate(
    (digits_X_test_RP, get_one_hot(km.predict(digits_X_test_RP), 10)),
    axis=1
)
digits_X_test_RP_gmm = np.concatenate(
    (digits_X_test_RP, get_one_hot(gmm.predict(digits_X_test_RP), 9)),
    axis=1
)

In [26]:
## RP + KMEANS
ann = MLPClassifier(
    activation='relu', max_iter=5000,
    solver='adam', learning_rate='adaptive', 
    hidden_layer_sizes=(500, 500), alpha= 0.01
)
start_time = timeit.default_timer()
ann.fit(digits_X_RP_km, digits_y)
predict = ann.predict(digits_X_test_RP_km)
print(classification_report(predict, digits_y_test))
end_time = timeit.default_timer()
elapsed = end_time - start_time
print("Done: {} seconds".format(elapsed))

             precision    recall  f1-score   support

          0       0.99      0.99      0.99       177
          1       0.97      0.89      0.93       199
          2       0.97      0.98      0.97       176
          3       0.92      0.95      0.94       176
          4       0.94      0.94      0.94       180
          5       0.97      0.94      0.95       189
          6       0.97      0.97      0.97       181
          7       0.95      0.98      0.97       173
          8       0.84      0.94      0.89       157
          9       0.96      0.92      0.94       189

avg / total       0.95      0.95      0.95      1797

Done: 23.628567872801796 seconds


In [27]:
## RP + GMM
ann = MLPClassifier(
    activation='relu', max_iter=5000,
    solver='adam', learning_rate='adaptive', 
    hidden_layer_sizes=(500, 500), alpha= 0.01
)
start_time = timeit.default_timer()
ann.fit(digits_X_RP_gmm, digits_y)
predict = ann.predict(digits_X_test_RP_gmm)
print(classification_report(predict, digits_y_test))
end_time = timeit.default_timer()
elapsed = end_time - start_time
print("Done: {} seconds".format(elapsed))

             precision    recall  f1-score   support

          0       0.99      0.99      0.99       178
          1       0.98      0.94      0.96       190
          2       0.97      0.98      0.98       175
          3       0.90      0.97      0.93       170
          4       0.97      0.96      0.96       184
          5       0.98      0.94      0.96       191
          6       0.98      0.97      0.98       182
          7       0.94      0.97      0.96       174
          8       0.91      0.89      0.90       177
          9       0.94      0.96      0.95       176

avg / total       0.96      0.96      0.96      1797

Done: 26.238323063822463 seconds


## Random Forest

In [28]:
rfc = RandomForestClassifier(
        n_estimators=100, class_weight='balanced', random_state=0)
fi = rfc.fit(digits_X, digits_y).feature_importances_
i = [i + 1 for i in range(len(fi))]
fi = pd.DataFrame({'importance': fi, 'feature': i})
fi.sort_values('importance', ascending=False, inplace=True)
fi['i'] = i
cumfi = fi['importance'].cumsum()

idxs = fi.loc[:cumfi.where(cumfi > 0.8).idxmin(), :]
idxs = list(idxs.index)

digits_X_RF = digits_X[:, idxs]
digits_X_test_RF = digits_X_test[:, idxs]

In [29]:
ann = MLPClassifier(
    activation='relu', max_iter=5000,
    solver='adam', learning_rate='adaptive', 
    hidden_layer_sizes=(500, 500), alpha= 0.01
)
start_time = timeit.default_timer()
ann.fit(digits_X_RF,digits_y)
predict = ann.predict(digits_X_test_RF)
print(classification_report(predict, digits_y_test))
end_time = timeit.default_timer()
elapsed = end_time - start_time
print("Done: {} seconds".format(elapsed))

             precision    recall  f1-score   support

          0       0.99      0.99      0.99       178
          1       0.98      0.96      0.97       185
          2       0.95      0.99      0.97       170
          3       0.97      0.97      0.97       184
          4       0.98      0.97      0.98       183
          5       0.98      0.93      0.95       191
          6       0.98      0.99      0.98       179
          7       0.91      0.99      0.95       164
          8       0.95      0.93      0.94       178
          9       0.97      0.95      0.96       185

avg / total       0.97      0.97      0.97      1797

Done: 21.53308790596202 seconds


In [30]:
km = KMeans(random_state=0)  # K-Means
gmm = GMM(random_state=0)  # Gaussian Mixture Model (EM)
km.set_params(n_clusters=11)
gmm.set_params(n_components=9)
km.fit(digits_X_RF)
gmm.fit(digits_X_RF)

digits_X_RF_km = np.concatenate(
    (digits_X_RF, get_one_hot(km.predict(digits_X_RF), 11)),
    axis=1
)
digits_X_RF_gmm = np.concatenate(
    (digits_X_RF, get_one_hot(gmm.predict(digits_X_RF), 9)),
    axis=1
)

digits_X_test_RF_km = np.concatenate(
    (digits_X_test_RF, get_one_hot(km.predict(digits_X_test_RF), 11)),
    axis=1
)
digits_X_test_RF_gmm = np.concatenate(
    (digits_X_test_RF, get_one_hot(gmm.predict(digits_X_test_RF), 9)),
    axis=1
)

In [31]:
## RP + KMEANS
ann = MLPClassifier(
    activation='relu', max_iter=5000,
    solver='adam', learning_rate='adaptive', 
    hidden_layer_sizes=(500, 500), alpha= 0.01
)
start_time = timeit.default_timer()
ann.fit(digits_X_RF_km, digits_y)
predict = ann.predict(digits_X_test_RF_km)
print(classification_report(predict, digits_y_test))
end_time = timeit.default_timer()
elapsed = end_time - start_time
print("Done: {} seconds".format(elapsed))

             precision    recall  f1-score   support

          0       0.99      1.00      1.00       177
          1       0.99      0.96      0.98       188
          2       0.96      0.99      0.98       171
          3       0.96      0.99      0.97       177
          4       0.99      0.97      0.98       184
          5       0.98      0.94      0.96       190
          6       0.98      1.00      0.99       178
          7       0.92      0.99      0.95       166
          8       0.96      0.92      0.94       182
          9       0.96      0.93      0.95       184

avg / total       0.97      0.97      0.97      1797

Done: 24.1700576399453 seconds


In [32]:
## RF + GMM
ann = MLPClassifier(
    activation='relu', max_iter=5000,
    solver='adam', learning_rate='adaptive', 
    hidden_layer_sizes=(500, 500), alpha= 0.01
)
start_time = timeit.default_timer()
ann.fit(digits_X_RF_gmm, digits_y)
predict = ann.predict(digits_X_test_RF_gmm)
print(classification_report(predict, digits_y_test))
end_time = timeit.default_timer()
elapsed = end_time - start_time
print("Done: {} seconds".format(elapsed))

             precision    recall  f1-score   support

          0       0.99      0.99      0.99       178
          1       0.99      0.95      0.97       189
          2       0.97      0.99      0.98       173
          3       0.95      0.99      0.97       176
          4       0.99      0.98      0.99       183
          5       0.98      0.94      0.96       190
          6       0.98      0.99      0.99       178
          7       0.92      0.98      0.95       168
          8       0.93      0.92      0.92       175
          9       0.97      0.93      0.95       187

avg / total       0.97      0.97      0.97      1797

Done: 21.191694607026875 seconds
